In [1]:
library(abind)
library(IRdisplay)
set.seed(1)
if (getwd() != "/projects/ps-gymreklab/amassara/simulate_gwas") {
    setwd("/projects/ps-gymreklab/amassara/simulate_gwas")
}
out = "out/1_98001984-99001984/susieR"
dir.create(out, showWarnings = FALSE)

In [2]:
n = 500
p = 1000
b = rep(0,p)
b[200] = 1
b[800] = 1
X = matrix(rnorm(n*p),nrow=n,ncol=p)
X[,200] = X[,400]
X[,600] = X[,800]
y = X %*% b + rnorm(n)

In [3]:
pdf('truth.pdf', width =5, height = 5, pointsize=16)
plot(b, col="black", pch=16, main = 'True effect size')
pos = 1:length(b)
points(pos[b!=0],b[b!=0],col=2,pch=16)
dev.off()

png 
  2

In [4]:
display_pdf(file = 'truth.pdf')

In [5]:
# alpha = 1
# y.fit = glmnet::glmnet(X,y,alpha = alpha,intercept = FALSE)
# y.cv  = glmnet::cv.glmnet(X,y,alpha = alpha,intercept = FALSE,
#                          lambda = y.fit$lambda)
# bhat  = glmnet::predict.glmnet(y.fit,type ="coefficients",
#                                s = y.cv$lambda.min)[-1,1]

In [6]:
# pdf('lasso.pdf', width =5, height = 5, pointsize=16)
# plot(bhat, col="black", pch=16, main = 'Lasso')
# pos = 1:length(bhat)
# points(pos[b!=0],bhat[b!=0],col=2,pch=16)   
# dev.off()

In [7]:
# display_pdf('lasso.pdf')

# Existing Bayesian methods for sparse regression

In [8]:
mm_regression = function(X, Y, Z=NULL) {
  if (!is.null(Z)) {
      Z = as.matrix(Z)
  }
  reg = lapply(seq_len(ncol(Y)), function (i) simplify2array(susieR:::univariate_regression(X, Y[,i], Z)))
  reg = do.call(abind, c(reg, list(along=0)))
  # return array: out[1,,] is betahat, out[2,,] is shat
  return(aperm(reg, c(3,2,1)))
}
sumstats = mm_regression(as.matrix(X), as.matrix(y))
dat = list(X=X,Y=as.matrix(y))
input = paste0(out,'/Toy.sumstats.rds')
saveRDS(list(data=dat, sumstats=sumstats), input)

In [9]:
output = paste0(out, "/Toy.N2finemapping.FINEMAP")
args = "--n-causal-snps 2"
commandArgs = function(...) 1

# source(paste0(.libPaths(), '/susieR/code/finemap.R'))
source("/projects/ps-gymreklab/amassara/simulate_gwas/temp/finemap.R")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [10]:
finemap = readRDS(paste0(out,"/Toy.N2finemapping.FINEMAP.rds"))[[1]]
head(finemap$set)

,rank,config,prob,log10bf,odds,k,prob_norm_k,h2,h2_0.95CI,mean,sd
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>
1,1,"200,600",2.5000e-01,122.1380,1,2,2.5000e-01,0.602946,"0.532836,0.677494","NA,NA","NA,NA"
2,2,"200,800",2.5000e-01,122.1380,1,2,2.5000e-01,0.602946,"0.532079,0.677991","NA,NA","NA,NA"
3,3,"400,600",2.5000e-01,122.1380,1,2,2.5000e-01,0.602946,"0.531776,0.683898","NA,NA","NA,NA"
4,4,"400,800",2.5000e-01,122.1380,1,2,2.5000e-01,0.602946,"0.53088,0.676159","NA,NA","NA,NA"
5,5,"200,768",2.2188e-66,57.0865,Inf,2,2.2188e-66,0.346559,"0.276517,0.419864","NA,NA","NA,NA"
6,6,"400,768",2.2188e-66,57.0865,Inf,2,2.2188e-66,0.346559,"0.271193,0.424564","NA,NA","NA,NA"


In [11]:
snp = finemap$snp
pip = snp[order(as.numeric(snp$snp)),]$snp_prob

In [32]:
pdf(paste0(out,'/Toy.finemap.pdf'), width =5, height = 5, pointsize=16)
susieR::susie_plot(pip, y='PIP', b=b, main = 'Bayesian sparse regression')
dev.off()

png 
  2

In [33]:
display_pdf(file=paste0(out,'/Toy.finemap.pdf'))

# SuSiE

In [34]:
fitted = susieR::susie(X, y, L=5,
               estimate_residual_variance=TRUE, 
               scaled_prior_variance=0.2,
               tol=1e-3, track_fit=TRUE, min_abs_corr=0.1)

In [35]:
pdf(paste0(out,'/Toy.susie.pdf'), width =5, height = 5, pointsize=16)
susieR::susie_plot(fitted, y='PIP', b=b, max_cs=0.4, main = paste('SuSiE, ', length(fitted$sets$cs), 'CS identified'))
dev.off()

png 
  2

In [36]:
display_pdf(file=paste0(out,'/Toy.susie.pdf'))

## SuSiE Effect Size Estimate

In [37]:
bhat = coef(fitted)[-1]
pdf(paste0(out,'/Toy.susie_eff.pdf'), width =5, height = 5, pointsize=16)
susieR::susie_plot(bhat, y='bhat', b=b, main = 'SuSiE, effect size estimate') 
dev.off()

png 
  2

In [38]:
display_pdf(file=paste0(out,'/Toy.susie_eff.pdf'))